In [ ]:

# import  os
# import pandas as pd
# directory = '/content/DATAset'

# files = os.listdir(directory)

# csv_files = [f for f in files if f.endswith('.csv')]

# dfs = []
# for csv_file in csv_files:
#     file_path = os.path.join(directory, csv_file)
#     df = pd.read_csv(file_path)
#     dfs.append(df)

# df = pd.concat(dfs, ignore_index=True)

In [ ]:
# df.to_csv(directory+'cleaned_dataset.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.cluster import DBSCAN
import numpy as np
from datetime import datetime, timedelta
import pytz
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/DATAsetcleaned_dataset.csv')

In [ ]:
df.head()

,road_name,distance_km,duration_in_traffic_min,speed_kmh,delay_min,traffic_condition,timestamp,start_point,end_point
0,Al Nadwah,10.482,21.500000,29.252093,1.033333,Yellow,2024-09-22T20:00:30.204384,"(24.8334879, 46.92008999999999)","(24.7888572, 46.8588979)"
1,Al Nadwah,2.397,5.966667,24.103911,-1.016667,Blue,2024-09-22T20:00:30.314064,"(24.7888572, 46.8588979)","(24.7743639, 46.8608065)"
2,King Abdul Aziz Road,12.235,19.600000,37.454082,-0.300000,Blue,2024-09-22T20:00:30.491019,"(24.999486, 46.72798419999999)","(24.9286095, 46.7150481)"
3,King Abdul Aziz Road,27.167,22.916667,71.128145,3.200000,Yellow,2024-09-22T20:00:30.604397,"(24.9286095, 46.7150481)","(24.796827, 46.5643251)"
4,King Abdul Aziz Road,17.492,13.383333,78.419925,1.100000,Yellow,2024-09-22T20:00:30.690591,"(24.796827, 46.5643251)","(24.6575642, 46.5630617)"


In [ ]:
df.shape

(3401, 9)

In [ ]:
df = df[df['delay_min'] > 0]

In [ ]:
df.tail()

,road_name,distance_km,duration_in_traffic_min,speed_kmh,delay_min,traffic_condition,timestamp,start_point,end_point
3232,الأشقر,6.918,9.700000,42.791753,0.100000,Blue,2024-09-23T09:00:31.375722,"(24.9089055, 46.9546838)","(24.8775235, 46.9313462)"
3313,طريق العروبة,2.665,3.783333,42.264317,0.116667,Blue,2024-09-23T09:00:38.920929,"(24.70444, 46.6237931)","(24.6894848, 46.6294245)"
3318,الدائري الثاني,2.949,3.483333,50.796172,0.150000,Blue,2024-09-23T09:00:39.388019,"(24.662313, 46.8567589)","(24.6525801, 46.8461368)"
3340,Maan Bin Ows,5.405,7.483333,43.336303,0.450000,Yellow,2024-09-23T09:00:41.474116,"(24.6319775, 46.8013918)","(24.6365608, 46.8002853)"
3400,الباطن,6.993,9.266667,45.278417,0.266667,Blue,2024-09-23T09:00:46.994943,"(24.6705, 46.7815592)","(24.6439747, 46.7303048)"


In [ ]:
df.shape

(557, 9)

In [ ]:
# Convert the 'timestamp' column to datetime objects first
df['timestamp'] = pd.to_datetime(df['timestamp'])
# Now you can add the timedelta

df['timestamp'] = pd.to_datetime(df['timestamp'] + timedelta(hours=3))

In [ ]:
df['timestamp'].sort_values()

,timestamp
895,2024-09-22 21:00:30.129180
898,2024-09-22 21:00:30.488185
899,2024-09-22 21:00:30.584245
900,2024-09-22 21:00:30.682290
902,2024-09-22 21:00:30.852108
...,...
2586,2024-09-23 15:00:37.791189
2597,2024-09-23 15:00:38.782471
2648,2024-09-23 15:00:43.609621
2654,2024-09-23 15:00:44.173050


In [ ]:
df['delay_min'].value_counts().max()

11

In [ ]:
# df[df['delay_min']>30]

In [ ]:
# df['delay_min'].value_counts().min()

In [ ]:
# Convert the 'timestamp' column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract the hour from the 'timestamp' column to identify the hour of congestion
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month

# Feature engineering: create a new column for congestion ratio
# Calculate the congestion ratio as (delay_min / distance_km)
df['congestion_ratio'] = df['delay_min'] / df['distance_km']

# Select relevant columns: road_name, distance_km, delay_min, congestion_ratio, hour, and other details
congestion_info = df[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'hour', 'traffic_condition', 'timestamp', 'start_point', 'end_point']]

# Display the top 10 most congested roads based on the congestion ratio
congestion_info.head(20)


,road_name,distance_km,delay_min,congestion_ratio,hour,traffic_condition,timestamp,start_point,end_point
0,Al Nadwah,10.482,1.033333,0.098582,23,Yellow,2024-09-22 23:00:30.204384,"(24.8334879, 46.92008999999999)","(24.7888572, 46.8588979)"
3,King Abdul Aziz Road,27.167,3.200000,0.117790,23,Yellow,2024-09-22 23:00:30.604397,"(24.9286095, 46.7150481)","(24.796827, 46.5643251)"
4,King Abdul Aziz Road,17.492,1.100000,0.062886,23,Yellow,2024-09-22 23:00:30.690591,"(24.796827, 46.5643251)","(24.6575642, 46.5630617)"
7,King Salman Road,23.308,4.700000,0.201648,23,Orange,2024-09-22 23:00:30.960832,"(24.9229714, 46.7204701)","(24.796827, 46.5643251)"
8,King Salman Road,5.568,0.266667,0.047893,23,Blue,2024-09-22 23:00:31.053344,"(24.796827, 46.5643251)","(24.796728, 46.5538512)"
13,العزايزه,11.967,3.133333,0.261831,23,Yellow,2024-09-22 23:00:31.526365,"(24.6131743, 46.8134172)","(24.5881304, 46.7601929)"
17,بنها,9.759,0.366667,0.037572,23,Blue,2024-09-22 23:00:31.851974,"(24.6113886, 46.8379204)","(24.5988688, 46.8215)"
18,تقي الدين التميمي,8.389,1.000000,0.119204,23,Yellow,2024-09-22 23:00:31.928555,"(24.663953, 46.5257629)","(24.6367317, 46.4820289)"
25,طريق خزام,10.312,0.433333,0.042022,23,Blue,2024-09-22 23:00:32.554812,"(25.0461094, 46.7471065)","(24.9628648, 46.7860364)"
27,طريق خزام,5.905,0.050000,0.008467,23,Blue,2024-09-22 23:00:32.704536,"(24.8918964, 46.7151761)","(24.8702366, 46.6964474)"


In [ ]:
df.shape

(557, 12)

In [ ]:
# Define the directory where you want to save the CSV file
directory = './' # Current directory

# Save the DataFrame to a CSV file in the specified directory
df.to_csv(directory+'cleaned(v2)_dataset.csv', index=False)

In [ ]:
# df['congestion_ratio'].max()

In [ ]:
# df['congestion_ratio'].min()

In [ ]:
# df= df[df['congestion_ratio'] >= 0]

In [ ]:
# # Prepare the feature for clustering (congestion_ratio)
# features = df[['congestion_ratio']]

# # Use MinMaxScaler to scale the congestion_ratio
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features)

# # Elbow Method to find the optimal number of clusters (k)
# inertia = []
# K = range(2, 20)  # Test different values of k (from 1 to 10)
# for k in K:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(features_scaled)
#     inertia.append(kmeans.inertia_)  # Inertia: sum of squared distances of samples to their closest cluster center

In [ ]:
# # Plot the Elbow Method
# plt.figure(figsize=(8, 5))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('Number of Clusters (k)')
# plt.ylabel('Inertia')
# plt.title('Elbow Method For Optimal k')
# plt.show()


In [ ]:
# # Prepare the feature for clustering (congestion_ratio)
# features = df[['congestion_ratio']]

# # Use MinMaxScaler to scale the congestion_ratio
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features)

# # Apply K-Means clustering with 5 clusters
# kmeans = KMeans(n_clusters=5, random_state=42)
# df['cluster'] = kmeans.fit_predict(features_scaled)

# # Display 10 examples from each cluster
# unique_clusters = df['cluster'].unique()

# for cluster in unique_clusters:
#     # Filter roads for the current cluster and print the first 10 examples
#     cluster_data = df[df['cluster'] == cluster].head(10)
#     print(f"\nCluster {cluster_data}:")
#     print("-----------------------------------")

# #cluster_data[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'traffic_condition', 'timestamp']]


In [ ]:
# df['cluster'].value_counts()

In [ ]:
# # Visualization of the clusters

# plt.figure(figsize=(8, 6))

# # Plot each cluster with different colors
# for cluster in unique_clusters:
#     cluster_points = df[df['cluster'] == cluster]
#     plt.scatter(cluster_points.index, cluster_points['congestion_ratio'], label=f"Cluster {cluster}")

# # Add labels and title
# plt.xlabel('Index (Roads)')
# plt.ylabel('Congestion Ratio')
# plt.title('K-Means Clustering Visualization (5 Clusters)')
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
# # Prepare the feature for clustering (congestion_ratio)
# features = df[['congestion_ratio']]

# # Use MinMaxScaler to scale the congestion_ratio
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features)

# # Apply K-Means clustering with 6 clusters
# kmeans = KMeans(n_clusters=6, random_state=42)
# df['cluster'] = kmeans.fit_predict(features_scaled)

# # Display 10 examples from each of the 6 clusters
# unique_clusters = df['cluster'].unique()

# # For each cluster, show the first 10 roads
# for cluster in unique_clusters:
#     # Filter roads for the current cluster
#     cluster_data = df[df['cluster'] == cluster].head(10)
#     print(f"\nCluster {cluster_data}:")
#     print("-----------------------------------")

# #cluster_data[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'traffic_condition', 'timestamp']]


In [ ]:
# df['cluster'].value_counts()

In [ ]:
# # Visualization of the clusters

# plt.figure(figsize=(8, 6))

# # Plot each cluster with different colors
# for cluster in unique_clusters:
#     cluster_points = df[df['cluster'] == cluster]
#     plt.scatter(cluster_points.index, cluster_points['congestion_ratio'], label=f"Cluster {cluster}")

# # Add labels and title
# plt.xlabel('Index (Roads)')
# plt.ylabel('Congestion Ratio')
# plt.title('K-Means Clustering Visualization (6 Clusters)')
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
# # Prepare the feature for clustering (congestion_ratio)
# features = df[['congestion_ratio']]
# # Use MinMaxScaler to scale the congestion_ratio
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features)

# # Apply K-Means clustering with 7 clusters
# kmeans = KMeans(n_clusters=7, random_state=42)
# df['cluster'] = kmeans.fit_predict(features_scaled)

# # Display 10 examples from each of the 7 clusters
# unique_clusters = df['cluster'].unique()

# # For each cluster, show the first 10 roads
# for cluster in unique_clusters:
#     # Filter roads for the current cluster
#     cluster_data = df[df['cluster'] == cluster].head(10)
#     print(f"\nCluster {cluster_data}:")
#     print("-----------------------------------")

# #cluster_data[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'traffic_condition', 'timestamp']]


In [ ]:
# df['cluster'].value_counts()

In [ ]:
# # Visualization of the clusters
# plt.figure(figsize=(8, 6))

# # Plot each cluster with different colors
# for cluster in unique_clusters:
#     cluster_points = df[df['cluster'] == cluster]
#     plt.scatter(cluster_points.index, cluster_points['congestion_ratio'], label=f"Cluster {cluster}")

# # Add labels and title
# plt.xlabel('Index (Roads)')
# plt.ylabel('Congestion Ratio')
# plt.title('K-Means Clustering Visualization (7 Clusters)')
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# # Prepare the features for clustering (distance, delay, congestion ratio)
# features = df[['distance_km', 'delay_min', 'congestion_ratio']]

# # Use MinMaxScaler instead of StandardScaler
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features)

# # Apply DBSCAN for density-based clustering (eps and min_samples can be adjusted)
# dbscan = DBSCAN(eps=0.5, min_samples=5)
# df['cluster'] = dbscan.fit_predict(features_scaled)

# # Filter out noise (-1 means noise points)
# clustered_roads = df[df['cluster'] != -1]

# # Sort by congestion ratio to identify the most congested and longest roads
# best_routes = clustered_roads.sort_values(by=['congestion_ratio'], ascending=[True])

# # Select relevant columns and display the top 10 most congested roads
# best_routes = best_routes[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'cluster', 'traffic_condition', 'timestamp', 'start_point', 'end_point']]

# # Display the top 10 unique, most congested roads with long distances
# best_routes.head(10)


In [ ]:
# # Prepare the feature for clustering (congestion_ratio)
# features = df[['congestion_ratio']]

# # Use MinMaxScaler to scale the congestion_ratio
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features)

# # Apply DBSCAN for density-based clustering (eps and min_samples can be adjusted)
# dbscan = DBSCAN(eps=0.2, min_samples=5)  # Adjust 'eps' based on density sensitivity
# df['cluster'] = dbscan.fit_predict(features_scaled)

# # Filter out noise (-1 means noise points)
# clustered_roads = df[df['cluster'] != -1]

# # Select relevant columns and display the top congested roads based on clusters
# best_routes = clustered_roads[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'cluster', 'traffic_condition', 'timestamp', 'start_point', 'end_point']]

# # Display the clusters of roads with high congestion ratio
# best_routes.head(20)


In [ ]:
# df['cluster'].value_counts()

In [ ]:
# # Display 10 examples from each cluster
# unique_clusters = clustered_roads['cluster'].unique()

# for cluster in unique_clusters:
#     # Filter roads for the current cluster
#     cluster_data = clustered_roads[clustered_roads['cluster'] == cluster].head(10)
#     print(f"\nCluster {cluster}:")
#     print(cluster_data[['road_name', 'distance_km', 'delay_min', 'congestion_ratio', 'traffic_condition', 'timestamp']])


In [ ]:
# import pandas as pd

# # Group by road_name and aggregate the total distance and count of traffic conditions
# crowded_streets = df.groupby('road_name').agg(
#     total_distance_km=('distance_km', 'sum'),  # Total distance of the street
#     avg_delay_min=('delay_min', 'mean'),  # Average delay per street (as an indicator of traffic)
#     traffic_condition_count=('traffic_condition', 'count')  # Total records per street (traffic instances)
# ).reset_index()

# # Sort by average delay or the count of traffic conditions to see the most crowded streets
# # You can change sorting criteria based on what defines 'most crowded'
# crowded_streets = crowded_streets.sort_values(by='avg_delay_min', ascending=False)

# # Display the result
# crowded_streets

In [ ]:
# pd.reset_option('display.max_rows', None)
# crowded_streets

In [ ]:
# # Convert the 'timestamp' column to datetime if it's not already
# df['timestamp'] = pd.to_datetime(df['timestamp'])

# # Extract the hour from the 'timestamp' column
# df['hour'] = df['timestamp'].dt.hour

# # Sort the data by road name, delay, distance, and hour
# df_sorted = df.sort_values(by=['road_name', 'delay_min', 'distance_km', 'hour'], ascending=[True, False, False, True])

# # Group by 'road_name' to collect the most crowded instance of each road based on delay_min
# most_crowded_roads = df_sorted.groupby('road_name').first().reset_index()

# # Sort the final result by delay to find the most crowded roads overall
# most_crowded_roads = most_crowded_roads.sort_values(by='delay_min', ascending=False)

# # Display the top 10 most crowded roads based on delay, time, and distance
# most_crowded_roads[['road_name', 'distance_km', 'delay_min', 'hour', 'traffic_condition', 'timestamp']].head(15)


In [ ]:
# # Plotting the top 10 most crowded roads based on delay and distance
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))

# # Select the top 10 most crowded roads based on delay
# top_crowded_roads = most_crowded_roads.head(15)

# # Plotting the bar chart
# plt.barh(top_crowded_roads['road_name'] + " " + top_crowded_roads['hour'].astype(str) + "h (" + top_crowded_roads['traffic_condition'] + ")", top_crowded_roads['delay_min'], color='orange')
# plt.xlabel('Delay (Minutes)')
# plt.ylabel('Road Name - Hour (Traffic Condition)')
# plt.title('Top 10 Most Crowded Roads by Delay, Time, and Distance')
# plt.tight_layout()

# # Display the plot
# plt.show()

In [ ]:
# # Convert the 'timestamp' column to datetime if it's not already
# df['timestamp'] = pd.to_datetime(df['timestamp'])

# # Extract the hour from the 'timestamp' column
# df['hour'] = df['timestamp'].dt.hour
# df['month'] = df['timestamp'].dt.month
# df['day'] = df['timestamp'].dt.day

# # Sort the data by road_name, delay_min, and distance_km to prioritize high delays with large distances
# df_sorted = df.sort_values(by=['road_name', 'delay_min', 'distance_km'], ascending=[True, False, False])

# # Group by 'road_name' to select the row with the highest delay and distance for each road
# #most_crowded_roads = df_sorted.groupby('road_name').first().reset_index()

# # Sort the final result by delay and distance to highlight the most congested roads
# df_sorted = df_sorted.sort_values(by=['delay_min', 'distance_km'], ascending=False)

# # Display the top 10 most crowded roads with the highest delay and distance
# df_sorted[['road_name', 'distance_km', 'delay_min', 'hour', 'traffic_condition', 'timestamp','start_point','end_point']].head(30)

In [ ]:
# # Plotting the top 10 most crowded roads based on both highest delay and distance
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))

# # Select the top 10 most crowded roads
# top_crowded_roads = df_sorted.head(10)

# # Plotting the bar chart
# plt.barh(df_sorted['road_name'] + " " + df_sorted['hour'].astype(str) + "h (" + df_sorted['traffic_condition'] + ")", df_sorted['delay_min'], color='orange')
# plt.xlabel('Delay (Minutes)')
# plt.ylabel('Road Name - Hour (Traffic Condition)')
# plt.title('Top 10 Most Congested Roads by Highest Delay and Distance')
# plt.tight_layout()

# # Display the plot
# plt.show()

In [ ]:
# # Prepare the numerical features: distance and delay
# features = df[['distance_km', 'delay_min']]

# # Prepare the categorical feature: traffic_condition
# # OneHotEncode the traffic_condition column
# encoder = OneHotEncoder(sparse=False)
# features1_encoded = encoder.fit_transform(df[['traffic_condition']])

# # Standardize the numerical features to ensure proper scaling
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

# # Combine the scaled numerical features with the one-hot encoded categorical features
# combined_features = pd.concat([pd.DataFrame(features_scaled), pd.DataFrame(features1_encoded)], axis=1)

# # Apply DBSCAN clustering
# # eps defines the maximum distance between two points to be considered neighbors
# # min_samples defines the minimum number of points to form a dense cluster
# dbscan = DBSCAN(eps=0.5, min_samples=5)
# df['cluster'] = dbscan.fit_predict(combined_features)

# # Filter out noise (points labeled as -1 by DBSCAN)
# congested_places = df[df['cluster'] != -1]

# # Sort by delay and distance to identify the most congested roads
# congested_places = congested_places.sort_values(by=['delay_min', 'distance_km'], ascending=[False, False])

# # Display the most congested roads directly
# congested_places[['road_name', 'distance_km', 'delay_min', 'hour', 'traffic_condition', 'cluster']].head(114)


In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))

# # Plotting distance vs. delay with color-coded clusters
# plt.scatter(congested_places['distance_km'], congested_places['delay_min'], c=congested_places['cluster'], cmap='rainbow')
# plt.xlabel('Distance (km)')
# plt.ylabel('Delay (Minutes)')
# plt.title('Congested Roads Identified by DBSCAN')
# plt.colorbar(label='Cluster')
# plt.show()
